#### Buiding pandas API documentation dataset as per following format:
#### [Category,  SubCategory, FullyQualifiedName, MethodName, Description]

##### Using API documentation from https://pandas.pydata.org/pandas-docs/stable/api.html

In [2]:
from bs4 import BeautifulSoup
import pandas as pd
import pickle

In [3]:
SEED_API_DOCUMENTATION_FILE = '../../../data/api_doc/pandas/API_Reference_pandas_0_22_0documentation.html'
API_DOCUMENTATION_CSV_FILE_NAME = 'api_doc.csv'

In [4]:
df = pd.DataFrame(columns=['Category', 'SubCategory', 'FullyQualifiedName', 'MethodName', 'Description'])
loc_id = 0
api_file = open(SEED_API_DOCUMENTATION_FILE, 'r')
api_html_content = api_file.read()
soup = BeautifulSoup(api_html_content, 'html.parser')
div_api_reference = soup.find("div", {"id": "api-reference"})
#children = div_api_reference.findChildren()
#for child in children:
    #print child
first_level_divs = div_api_reference.find_all("div", {"class": "section"}, recursive=False)

for first_level_div in first_level_divs:
    
    #Get the category
    #Text comes in unicode string form. Coverting it to python string form
    category = first_level_div.find("h2").text.encode('ascii', 'ignore')
    second_level_divs = first_level_div.find_all("div", {"class": "section"}, recursive=False)
    for second_level_div in second_level_divs:
        subcategory = second_level_div.find("h3").text.encode('ascii', 'ignore')
        table_rows = second_level_div.find("table").find("tbody").find_all("tr")
        for r in table_rows:
            table_datas = r.find_all("td")
            fully_qualified_name = table_datas[0].find("a")['title'].encode('ascii', 'ignore')
            description = table_datas[1].text.encode('ascii', 'ignore')
            method_name = fully_qualified_name[fully_qualified_name.rfind(".")+1:].encode('ascii', 'ignore')
            df_row = [category, subcategory, fully_qualified_name, method_name, description]
            df.loc[loc_id] = df_row
            loc_id = loc_id + 1
df.to_csv(API_DOCUMENTATION_CSV_FILE_NAME)

In [5]:
def save_object(obj, filename):
    with open(filename, 'wb') as output:  # Overwrites any existing file.
        pickle.dump(obj, output, pickle.HIGHEST_PROTOCOL)

In [6]:
save_object(df, 'api_doc.pkl')